In [1]:
import numpy as np
from PIL import Image
import cv2 #opencv
import io
import time
import pandas as pd
import numpy as np
from IPython.display import clear_output
from random import randint
import os

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

#keras imports
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
from keras.callbacks import TensorBoard
from collections import deque
import random
import pickle
from io import BytesIO
import base64
import json

In [2]:
#path variables
chrome_driver_path = "C:/Users/ARDA ARAS/Desktop/DinoRunTutorial-master/chromedriver.exe"
loss_file_path = "./objects/loss_df.csv"
actions_file_path = "./objects/actions_df.csv"
q_value_file_path = "./objects/q_values.csv"
scores_file_path = "./objects/scores_df.csv"

init_script = "document.getElementsByClassName('runner-canvas')[0].id = 'runner-canvas'"

getbase64Script = "canvasRunner = document.getElementById('runner-canvas'); \
return canvasRunner.toDataURL().substring(22)"

In [3]:
class Game:
    def __init__(self,custom_config=True):
        chrome_options = Options()
        chrome_options.add_argument("disable-infobars")
        chrome_options.add_argument("--mute-audio")
        self._driver = webdriver.Chrome(executable_path = chrome_driver_path,chrome_options=chrome_options)
        self._driver.get('http://www.trex-game.skipser.com/')
        self._driver.execute_script("Runner.config.ACCELERATION=0")
        self._driver.execute_script(init_script)
    def get_crashed(self):
        return self._driver.execute_script("return Runner.instance_.crashed")
    def get_playing(self):
        return self._driver.execute_script("return Runner.instance_.playing")
    def restart(self):
        self._driver.execute_script("Runner.instance_.restart()")
    def press_up(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_UP)
    def press_down(self):
        self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
#         endtime = time.time() + 1.0
#         while True:
#             if time.time() > endtime:
#                 self._driver.find_element_by_tag_name("body").send_keys(Keys.ARROW_DOWN)
#                 break
    def get_score(self):
        score_array = self._driver.execute_script("return Runner.instance_.distanceMeter.digits")
        score = ''.join(score_array) 
        return int(score)
    def pause(self):
        return self._driver.execute_script("return Runner.instance_.stop()")
    def resume(self):
        return self._driver.execute_script("return Runner.instance_.play()")
    def end(self):
        self._driver.close()

In [4]:
class DinoAgent:
    def __init__(self,game): 
        self._game = game; 
        self.jump(); 
    def is_running(self):
        return self._game.get_playing()
    def is_crashed(self):
        return self._game.get_crashed()
    def jump(self):
        self._game.press_up()
    def duck(self):
        self._game.press_down()

In [5]:
class Game_state:
    def __init__(self,agent,game):
        self._agent = agent
        self._game = game
        self._display = show_img()  
        self._display.__next__()  
    def get_state(self,actions):
        
        score = self._game.get_score() 
        reward = 0.1
        is_over = False #game over
        if actions[1] == 1:
            self._agent.jump()
            actions_df.loc[len(actions_df)] = 1 
        elif actions[2] == 1:
            self._agent.duck()
            actions_df.loc[len(actions_df)] = 2 
        else:
            actions_df.loc[len(actions_df)] = 0 
        image = grab_screen(self._game._driver) 
        self._display.send(image) 
        if self._agent.is_crashed():
            scores_df.loc[len(loss_df)] = score 
            self._game.restart()
            reward = -1
            is_over = True
        return image, reward, is_over 


In [6]:
def save_obj(obj, name ):
    with open('objects/'+ name + '.pkl', 'wb') as f: 
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
def load_obj(name ):
    with open('objects/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

def grab_screen(_driver):
    image_b64 = _driver.execute_script(getbase64Script)
    screen = np.array(Image.open(BytesIO(base64.b64decode(image_b64))))
    image = process_img(screen)
    return image

def process_img(image):
    
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    image = image[:300, :500] 
    image = cv2.resize(image, (80,80))
    return  image

def show_img(graphs = False):
    while True:
        screen = (yield)
        window_title = "logs" if graphs else "game_play"
        cv2.namedWindow(window_title, cv2.WINDOW_NORMAL)        
        imS = cv2.resize(screen, (800, 400)) 
        cv2.imshow(window_title, screen)
        if (cv2.waitKey(1) & 0xFF == ord('q')):
            cv2.destroyAllWindows()
            break

In [7]:
#Intialize log structures from file if exists else create new
loss_df = pd.read_csv(loss_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns =['loss'])
scores_df = pd.read_csv(scores_file_path) if os.path.isfile(loss_file_path) else pd.DataFrame(columns = ['scores'])
actions_df = pd.read_csv(actions_file_path) if os.path.isfile(actions_file_path) else pd.DataFrame(columns = ['actions'])
q_values_df =pd.read_csv(actions_file_path) if os.path.isfile(q_value_file_path) else pd.DataFrame(columns = ['qvalues'])

In [8]:
#game parameters
ACTIONS = 3 
GAMMA = 0.99 
OBSERVATION = 100. 
EXPLORE = 100000  
FINAL_EPSILON = 0.0001 
INITIAL_EPSILON = 0.1 
REPLAY_MEMORY = 50000 
BATCH = 16 
FRAME_PER_ACTION = 1
LEARNING_RATE = 1e-4
img_rows , img_cols = 80,80
img_channels = 4 

In [9]:
def buildmodel():
    print("We build the model")
    model = Sequential()
    model.add(Conv2D(32, (8, 8), padding='same',strides=(4, 4),input_shape=(img_cols,img_rows,img_channels)))  #80*80*4
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4),strides=(2, 2),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3),strides=(1, 1),  padding='same'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Activation('relu'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS))
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)

    model.save_weights('model.h5')
    print("Model is finished")
    return model

In [10]:
def stack_image(image, times):
    stacked_image = list()
    for i in range(times):
        stacked_image.append(image)
    stacked_image = np.array(stacked_image)
    return stacked_image

In [11]:
def trainNetwork(model,game_state,observe=False):

    D = load_obj("D") 
    
    do_nothing = np.array([1,0,0])
    
    x_t, r_0, terminal = game_state.get_state(do_nothing) 
    
    s_t = stack_image(x_t, 4)
    s_t = s_t.reshape(1, s_t.shape[1], s_t.shape[2], -1)
 
    initial_state = s_t 
    
    OBSERVE = OBSERVATION
    epsilon = load_obj("epsilon") 
    model.load_weights("model.h5")
    adam = Adam(lr=LEARNING_RATE)
    model.compile(loss='mse',optimizer=adam)
    t = load_obj("time") 
    while (True):
        
        loss = 0
        Q_sa = 0
        action_index = 0
        r_t = 0 
        a_t = np.zeros([ACTIONS]) 
        
        cons = 0.75
        if  random.random() > epsilon: 
            print("Q values used")
            q = model.predict(s_t)       
            action_index = np.argmax(q)         
            a_t[action_index] = 1        
                
        else:
            print("Random Action")
            if random.random() <= cons:
                a_t[1] = 1
            else:
                action_index = random.randrange(ACTIONS)
                a_t[action_index] = 1
            

        x_t1, r_t, terminal = game_state.get_state(a_t)

        x_t1 = x_t1.reshape(1, x_t1.shape[0], x_t1.shape[1], 1)
        s_t1 = np.append(x_t1, s_t[:, :, :, :3], axis=3) 
        
        D.append((s_t, action_index, r_t, s_t1, terminal))
        
        if np.shape(D)[0] > REPLAY_MEMORY:
            D = D[1:]
            
        if t > OBSERVE: 
                                         
            batch_size = BATCH
            
            minibatch = random.sample(D, BATCH)
            shapes_states = np.shape(s_t)[1:]
            
            inputs = np.zeros((batch_size, 80,80,4))   
            targets = np.zeros((batch_size, ACTIONS))  

            if epsilon > FINAL_EPSILON:
                epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE
            
            for i in range(np.shape(minibatch)[0]):
                state_t, action_t, reward_t, state_t1, terminal  = minibatch[i]                

                inputs[i] = state_t    
                targets[i] = model.predict(state_t)
                Q_sa = model.predict(state_t1)      
                
                if not terminal:
                    targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa)
                else:
                    targets[i, action_t] = reward_t

            loss_batch = model.train_on_batch(inputs, targets)
            loss = loss + loss_batch 
            
            length_loss_df = np.shape(loss_df)[0]
            length_q_values_df = np.shape(q_values_df)[0]
            
            loss_df.loc[length_loss_df] = loss
            q_values_df.loc[length_q_values_df] = np.max(Q_sa)
        
        if not terminal:
            s_t = s_t1
        else:
            s_t = initial_state
        
        t = t + 1

In [12]:
#main function
def playGame(observe=False):
    game = Game()
    dino = DinoAgent(game)
    game_state = Game_state(dino,game)    
    model = buildmodel()
    try:
        trainNetwork(model,game_state,observe=observe)
    except StopIteration:
        game.end()

In [13]:
playGame(observe=False);

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


We build the model
Model is finished
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Random Action
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Q values used
Random Action
Q values used
Q values us

KeyboardInterrupt: 